In [37]:
import numpy as np
import pandas as pd
import os

# competition_folder = '/kaggle/input/cafa-6-protein-function-prediction/'
competition_folder = 'data'

list_of_filenames = []
for dirname, _, filenames in os.walk(competition_folder):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        list_of_filenames.append(filepath)
        print(filepath)

data\IA.tsv
data\sample_submission.tsv
data\Test\testsuperset-taxon-list.tsv
data\Test\testsuperset.fasta
data\Train\go-basic.obo
data\Train\train_sequences.fasta
data\Train\train_taxonomy.tsv
data\Train\train_terms.tsv


-------
-------
-------
# Observe File Contents (Terminal)
First, let's have a look at the first few lines of each file to know what we are dealing with.

In [38]:
train_sequences_file = competition_folder + '/Train/train_sequences.fasta'
!head {train_sequences_file} -n 10

>sp|A0A0C5B5G6|MOTSC_HUMAN Mitochondrial-derived peptide MOTS-c OS=Homo sapiens OX=9606 GN=MT-RNR1 PE=1 SV=1
MRWQEMGYIFYPRKLR
>sp|A0JNW5|BLT3B_HUMAN Bridge-like lipid transfer protein family member 3B OS=Homo sapiens OX=9606 GN=BLTP3B PE=1 SV=2
MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVLQNMLDLPTWLAINK
VFCNKASIRIPWTKLKTHPICLSLDKVIMEMSTCEEPRSPNGPSPIATASGQSEYGFAEK
VVEGISVSVNSIVIRIGAKAFNASFELSQLRIYSVNAHWEHGDLRFTRIQDPQRGEVLTF
KEINWQMIRIEADATQSSHLEIMCAPVRLITNQSKIRVTLKRRLKDCNVIATKLVLILDD
LLWVLTDSQLKAMVQYAKSLSEAIEKSTEQRKSMAPEPTQSSTVVASAQQVKTTQTSNAP
DVNDAIVKLFNDFDVKETSHHLVISHLDLHICDDIHAKEKESNRRITGGAMQLSFTQLTI
DYYPYHKAGDSCNHWMYFSDATKTKNGWANELLHEFECNVEMLKQAVKDHNVGSPPKSPT


In [39]:
train_terms_file = competition_folder + '/Train/train_terms.tsv'
!head {train_terms_file} -n 10

EntryID	term	aspect
Q5W0B1	GO:0000785	C
Q5W0B1	GO:0004842	F
Q5W0B1	GO:0051865	P
Q5W0B1	GO:0006275	P
Q5W0B1	GO:0006513	P
Q5W0B1	GO:0003682	F
Q5W0B1	GO:0005515	F
Q3EC77	GO:0000138	C
Q3EC77	GO:0005794	C


In [40]:
train_taxonomy_file = competition_folder + '/Train/train_taxonomy.tsv'
!head {train_taxonomy_file} -n 10

A0A0C5B5G6	9606
A0JNW5	9606
A0JP26	9606
A0PK11	9606
A1A4S6	9606
A1A519	9606
A1L190	9606
A1L3X0	9606
A1X283	9606
A2A2Y4	9606


In [41]:
go_file = competition_folder + '/Train/go-basic.obo'
!head {go_file} -n 55

format-version: 1.2
data-version: releases/2025-06-01
subsetdef: chebi_ph7_3 "Rhea list of ChEBI terms representing the major species at pH 7.3."
subsetdef: gocheck_do_not_annotate "Term not to be used for direct annotation"
subsetdef: gocheck_obsoletion_candidate "Terms planned for obsoletion"
subsetdef: goslim_agr "AGR slim"
subsetdef: goslim_aspergillus "Aspergillus GO slim"
subsetdef: goslim_candida "Candida GO slim"
subsetdef: goslim_chembl "ChEMBL protein targets summary"
subsetdef: goslim_drosophila "Drosophila GO slim"
subsetdef: goslim_euk_cellular_processes_ribbon "GO ribbon for eukaroytic cellular processes"
subsetdef: goslim_flybase_ribbon "Insecta GO ribbon slim"
subsetdef: goslim_generic "Generic GO slim"
subsetdef: goslim_metagenomics "Metagenomics GO slim"
subsetdef: goslim_mouse "Mouse GO slim"
subsetdef: goslim_pir "PIR GO slim"
subsetdef: goslim_plant "Plant GO slim"
subsetdef: goslim_plant_ribbon "Plant GO ribbon"
subsetdef: goslim_pombe "Fission yeast GO slim"
subs

In [42]:
testsuperset_file = competition_folder + '/Test/testsuperset.fasta'
!head {testsuperset_file} -n 20

>A0A0C5B5G6 9606
MRWQEMGYIFYPRKLR
>A0A1B0GTW7 9606
MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKSRSSSLTLPSSRDPQ
PLRIQSCYLGDHISDGAWDPEGEGMRGGSRALAAVREATQRIQAVLAVQGPLLLSRDPAQ
YCHAVWGDPDSPNYHRCSLLNPGYKGESCLGAKIPDTHLRGYALWPEQGPPQLVQPDGPG
VQNTDFLLYVRVAHTSKCHQETVSLCCPGWSTAAQSQLTAALTSWAQRRGFVMLPRLCLK
LLGSSNLPTLASQSIRITGPSVIAYAACCQLDSEDRPLAGTIVYCAQHLTSPSLSHSDIV
MATLHELLHALGFSGQLFKKWRDCPSGFSVRENCSTRQLVTRQDEWGQLLLTTPAVSLSL
AKHLGVSGASLGVPLEEEEGLLSSHWEARLLQGSLMTATFDGAQRTRLDPITLAAFKDSG
WYQVNHSAAEELLWGQGSGPEFGLVTTCGTGSSDFFCTGSGLGCHYLHLDKGSCSSDPML
EGCRMYKPLANGSECWKKENGFPAGVDNPHGEIYHPQSRCFFANLTSQLLPGDKPRHPSL
TPHLKEAELMGRCYLHQCTGRGAYKVQVEGSPWVPCLPGKVIQIPGYYGLLFCPRGRLCQ
TNEDINAVTSPPVSLSTPDPLFQLSLELAGPPGHSLGKEQQEGLAEAVLEALASKGGTGR
CYFHGPSITTSLVFTVHMWKSPGCQGPSVATLHKALTLTLQKKPLEVYHGGANFTTQPSK
LLVTSDHNPSMTHLRLSMGLCLMLLILVGVMGTTAYQKRATLPVRPSASYHSPELHSTRV
PVRGIREV
>A0JNW5 9606
MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVLQNMLDLPTWLAINK
VFCNKASIRIPWTKLKTHPICLSLDKVIMEMSTCEEPRSPNGPSPIATASGQSEYGFAEK


In [43]:
testsuperset_taxon_file = competition_folder + '/Test/testsuperset-taxon-list.tsv'
!head {testsuperset_taxon_file} -n 10

ID	Species
9606	Homo sapiens
10116	Rattus norvegicus
39947	Oryza sativa subsp. japonica
7955	Danio rerio
7227	Drosophila melanogaster
6239	Caenorhabditis elegans
3702	Arabidopsis thaliana
10090	Mus musculus
9913	Bos taurus


In [44]:
ia_file = competition_folder + '/IA.tsv'
!head {ia_file} -n 10

GO:0000001	0.0
GO:0000002	2.8496657269155685
GO:0000011	0.13750352374993496
GO:0000012	6.038630248164372
GO:0000017	0.5145731728297583
GO:0000018	1.8796369862750815
GO:0000019	3.584962500721156
GO:0000022	0.0
GO:0000023	3.3115861506659034
GO:0000024	0.0


In [45]:
sample_submission_file = competition_folder + '/sample_submission.tsv'
!head {sample_submission_file} -n 20

A0A0C5B5G6	GO:0000001	0.123
A0A0C5B5G6	GO:0000002	0.456
A0A0C5B5G6	Text	0.123	Regulates insulin sensitivity and metabolic homeostasis
A0A0C5B5G6	Text	0.456	Inhibits the folate cycle, thereby reducing de novo purine biosynthesis which leads to the accumulation of the de novo purine synthesis intermediate 5-aminoimidazole-4-carboxamide (AICAR)
A0A0C5B5G6	Text	0.456	and the activation of the metabolic regulator 5-AMP-activated protein kinase (AMPK)
A0A1B0GTW7	GO:0000001	0.123
A0A1B0GTW7	GO:0000002	0.456
A0A1B0GTW7	Text	0.123	Putative metalloproteinase that plays a role in left-right patterning process
A0JNW5	GO:0000001	0.123
A0JNW5	GO:0000002	0.456
A0JP26	GO:0000001	0.123
A0JP26	GO:0000002	0.456
A0PK11	GO:0000001	0.123
A0PK11	GO:0000002	0.456
A1A4S6	GO:0000001	0.123
A1A4S6	GO:0000002	0.456
A1A519	GO:0000001	0.123
A1A519	GO:0000002	0.456
A1L190	GO:0000001	0.123
A1L190	GO:0000002	0.456


-------
-------
-------
# Install Necessary Libraries

In [46]:
# !pip install Bio fair-esm goatools

-------
-------
-------
# Imports

In [47]:
from Bio import SeqIO
from goatools.obo_parser import GODag
import random

-------
-------
-------
# EDA
Get some stats on the data, and observe some examples.

### Species Lookup

In [48]:
# species lookup table

species_lookup_df = pd.read_csv(testsuperset_taxon_file, sep="\t")
species_lookup_df.columns = ['species_id', 'species_name']
species_lookup_df

,species_id,species_name
0,9606,Homo sapiens
1,10116,Rattus norvegicus
2,39947,Oryza sativa subsp. japonica
3,7955,Danio rerio
4,7227,Drosophila melanogaster
...,...,...
8448,42344,Haliotis discus hannai
8449,8374,Crinia georgiana
8450,309541,Takifugu obscurus
8451,1274933,Schizocosa malitiosa


### Gene Ontology DAG

In [49]:
ontology = GODag(go_file)

# Example: Look up a GO term
term = ontology["GO:0006915"]
print('term.name:', term.name)
print('term.namespace:', term.namespace)
# print(term.parents)
# print(term.children)
# print(term.level)
# print(term.depth)
# print(term.is_obsolete)
# print(term.alt_ids)
# print(term.is_a)

# for documentation for the GoTerm class, refer to:
# https://github.com/tanghaibao/goatools/blob/main/goatools/obo_parser.py#L157

data/Train/go-basic.obo: fmt(1.2) rel(2025-06-01) 43,448 Terms
term.name: apoptotic process
term.namespace: biological_process


In [50]:
# number of terms in the GO graph

len(ontology)

43448

### Training Set FASTA Sequences (to be converted to features later)

In [51]:
# number of FASTA sequences in train & test sets

train_fasta_sequences = [{"id": record.id, "seq": record.seq} for record in SeqIO.parse(train_sequences_file, "fasta")]
test_fasta_sequences  = [{"id": record.id, "seq": record.seq} for record in SeqIO.parse(testsuperset_file,  "fasta")]
print(f"len(train_fasta_sequences) = {len(train_fasta_sequences)}")
print(f"len(test_fasta_sequences)  = {len(test_fasta_sequences)}")

len(train_fasta_sequences) = 82404
len(test_fasta_sequences)  = 224309


In [52]:
# look at data sample (first 3 examples)

for record in train_fasta_sequences[:3]:
    print(record['id'])
    print(record['seq'])
    print('')

sp|A0A0C5B5G6|MOTSC_HUMAN
MRWQEMGYIFYPRKLR

sp|A0JNW5|BLT3B_HUMAN
MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVLQNMLDLPTWLAINKVFCNKASIRIPWTKLKTHPICLSLDKVIMEMSTCEEPRSPNGPSPIATASGQSEYGFAEKVVEGISVSVNSIVIRIGAKAFNASFELSQLRIYSVNAHWEHGDLRFTRIQDPQRGEVLTFKEINWQMIRIEADATQSSHLEIMCAPVRLITNQSKIRVTLKRRLKDCNVIATKLVLILDDLLWVLTDSQLKAMVQYAKSLSEAIEKSTEQRKSMAPEPTQSSTVVASAQQVKTTQTSNAPDVNDAIVKLFNDFDVKETSHHLVISHLDLHICDDIHAKEKESNRRITGGAMQLSFTQLTIDYYPYHKAGDSCNHWMYFSDATKTKNGWANELLHEFECNVEMLKQAVKDHNVGSPPKSPTHASPQHTQTEKDYPLKGTCRTPSVLSQQSKAKLMSSSVVVRLADFNIYQVSTAEQCRSSPKSMICCNKKSLYLPQEMSAVYIEFTEYYYPDGKDFPIPSPNLYSQLNALQFTVDERSILWLNQFLLDLKQSLNQFMAVYKLNDNSKSDEHVDVRVDGLMLKFVIPSEVKSECHQDQPRAISIQSSEMIATNTRHCPNCRHSDLEALFQDFKDCDFFSKTYTSFPKSCDNFNLLHPIFQRHAHEQDTKMHEIYKGNITPQLNKNTLKTSAATDVWAVYFSQFWIDYEGMKSGKGRPISFVDSFPLSIWICQPTRYAESQKEPQTCNQVSLNTSQSESSDLAGRLKRKKLLKEYYSTESEPLTNGGQKPSSSDTFFRFSPSSSEADIHLLVHVHKHVSMQINHYQYLLLLFLHESLILLSENLRKDVEAVTGSPASQTSICIGILLRSAELALLLHPVDQANTLKSPVSESVSPVVPDYLPTENGDFLSSKRKQISRDINRIRSVTVNHMSDNRSM

### Training Set Terms (labels to be predicted)

In [53]:
# load training set terms

train_terms_df = pd.read_csv(train_terms_file, sep="\t")
train_terms_df

,EntryID,term,aspect
0,Q5W0B1,GO:0000785,C
1,Q5W0B1,GO:0004842,F
2,Q5W0B1,GO:0051865,P
3,Q5W0B1,GO:0006275,P
4,Q5W0B1,GO:0006513,P
...,...,...,...
537022,Q06667,GO:0070481,P
537023,B1NF19,GO:0033075,P
537024,B1NF19,GO:0047052,F
537025,B1NF19,GO:0047056,F


In [54]:
# number of unique values in each column

print(f"Number of unique entries: {train_terms_df.EntryID.nunique()}")
print(f"Number of unique terms:   {train_terms_df.term.nunique()}")
print(f"Number of unique aspects: {train_terms_df.aspect.nunique()}")

Number of unique entries: 82404
Number of unique terms:   26125
Number of unique aspects: 3


**From the cell above, It seems that what we have here is a *"multi-label classification problem"* involving 26,125 labels!**

That is, for each of the FASTA sequences, we are trying to predict which of the labels apply to it.

In [55]:
# total number of terms + number of unique terms per aspect

print('Total number of terms:', len(train_terms_df))
print('')
print(train_terms_df['aspect'].value_counts())

Total number of terms: 537027

aspect
P    250805
C    157770
F    128452
Name: count, dtype: int64


In [56]:
# NO intersections between terms of different aspects (as expected)

p_terms = train_terms_df[train_terms_df['aspect'] == "P"].term
c_terms = train_terms_df[train_terms_df['aspect'] == "C"].term
f_terms = train_terms_df[train_terms_df['aspect'] == "F"].term

print(set(p_terms).intersection(set(c_terms)))  #
print(set(p_terms).intersection(set(f_terms)))  # empty sets
print(set(c_terms).intersection(set(f_terms)))  #

set()
set()
set()


In [57]:
# number of terms per training sequence on average  ==>  ~6.5

train_terms_df.value_counts('EntryID').describe()

count    82404.000000
mean         6.517002
std          7.965655
min          1.000000
25%          2.000000
50%          4.000000
75%          8.000000
max        233.000000
Name: count, dtype: float64

### GO Graph Filtered by GO Terms found in Training Set

In [58]:
# keep elements of GO graph that also exist in training set

list_of_train_terms = list(train_terms_df['term'])
ontology_filtered = {k: ontology[k] for k in list_of_train_terms}
len(ontology_filtered)

26125

In [59]:
# look at an example terms and its parents

# random.seed(12345)
# random_go_term = random.sample(list(ontology_filtered.keys()), 1)[0]
random_go_term = 'GO:0043371'
random_go_term_parents = ontology_filtered[random_go_term].get_all_parents()

print(ontology_filtered[random_go_term])
print('')
print('Parents:')
for parent in random_go_term_parents:
    print(ontology[parent])

GO:0043371	level-10	depth-12	negative regulation of CD4-positive, alpha-beta T cell differentiation [biological_process]

Parents:
GO:0050863	level-06	depth-07	regulation of T cell activation [biological_process]
GO:0002694	level-04	depth-05	regulation of leukocyte activation [biological_process]
GO:0002683	level-04	depth-04	negative regulation of immune system process [biological_process]
GO:0050789	level-02	depth-02	regulation of biological process [biological_process]
GO:1903707	level-05	depth-07	negative regulation of hemopoiesis [biological_process]
GO:0045620	level-07	depth-09	negative regulation of lymphocyte differentiation [biological_process]
GO:0050865	level-04	depth-04	regulation of cell activation [biological_process]
GO:0050793	level-03	depth-03	regulation of developmental process [biological_process]
GO:0046636	level-08	depth-09	negative regulation of alpha-beta T cell activation [biological_process]
GO:0043370	level-09	depth-11	regulation of CD4-positive, alpha-beta T c

### Training Set Taxonomies (Species)

In [60]:
# species id for each fasta record in the train set

train_species_df = pd.read_csv(train_taxonomy_file, sep="\t", header=None)
train_species_df.columns = ['fasta_id', 'species_id']
train_species_df

,fasta_id,species_id
0,A0A0C5B5G6,9606
1,A0JNW5,9606
2,A0JP26,9606
3,A0PK11,9606
4,A1A4S6,9606
...,...,...
82399,Q9UTM1,284812
82400,Q9Y7I1,284812
82401,Q9Y7P7,284812
82402,Q9Y7Q3,284812


In [71]:
train_species_df = pd.merge(train_species_df, species_lookup_df, on='species_id', how='inner')
train_species_df[['species_id', 'species_name']].value_counts()

species_id  species_name                                         
9606        Homo sapiens                                             17162
10090       Mus musculus                                             12508
3702        Arabidopsis thaliana                                     11863
559292      Saccharomyces cerevisiae (strain ATCC 204508 / S288c)     5520
10116       Rattus norvegicus                                         4909
                                                                     ...  
2782196     Agkistrodon contortrix laticinctus                           1
2747968     Fusarium vanettenii                                          1
2718978     Rhopalurus crassicauda                                       1
2684170     Tityus cerroazul                                             1
2653441     Mucor thermohyalospora                                       1
Name: count, Length: 1381, dtype: int64